# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [15]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [43]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,20.46,56,0,3.60,AR,1680369129
1,1,waddan,29.1614,16.1390,25.65,9,0,2.34,LY,1680369129
2,2,san patricio,28.0170,-97.5169,28.98,57,97,3.25,US,1680369129
3,3,albany,42.6001,-73.9662,18.65,79,86,2.24,US,1680368994
4,4,hermanus,-34.4187,19.2345,19.33,73,0,7.14,ZA,1680369129


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City"
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [46]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[(city_data_df["Max Temp"] < 30 ) & 
                                (city_data_df["Max Temp"] >= 20) & 
                                (city_data_df["Wind Speed"] < 7) & 
                                (city_data_df["Cloudiness"] < 8) &
                                (city_data_df["Humidity"] < 70)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mar del plata,-38.0023,-57.5575,20.46,56,0,3.60,AR,1680369129
1,1,waddan,29.1614,16.1390,25.65,9,0,2.34,LY,1680369129
148,148,santa maria,-29.6842,-53.8069,22.87,30,0,2.38,BR,1680368898
159,159,los llanos de aridane,28.6585,-17.9182,22.86,41,0,5.66,ES,1680369152
233,233,esna,25.2934,32.5540,22.49,15,0,4.42,EG,1680369158


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[{"City", "Country", "Lat", "Lng", "Humidity"}].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

C:\Users\Kirby\AppData\Local\Temp\ipykernel_27332\2625336119.py:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  hotel_df = ideal_weather_df[{"City", "Country", "Lat", "Lng", "Humidity"}].copy()


,Country,Lat,Lng,City,Humidity,Hotel Name
0,AR,-38.0023,-57.5575,mar del plata,56,
1,LY,29.1614,16.1390,waddan,9,
148,BR,-29.6842,-53.8069,santa maria,30,
159,ES,28.6585,-17.9182,los llanos de aridane,41,
233,EG,25.2934,32.5540,esna,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 20  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mar del plata - nearest hotel: Nuevo Ostende
waddan - nearest hotel: No hotel found
santa maria - nearest hotel: Hotel Morotin
los llanos de aridane - nearest hotel: Valle Aridane
esna - nearest hotel: No hotel found
agadir - nearest hotel: Atlantic
broome - nearest hotel: No hotel found
chabahar - nearest hotel: هتل سپیده
loikaw - nearest hotel: Nawaday Hotel
altar - nearest hotel: Hotel Altar Inn
sao filipe - nearest hotel: Bela Vista
sabha - nearest hotel: فندق الكاظم-سبها
agua dulce - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
mecca - nearest hotel: Rehab Al Marwa Hotel 1
talcahuano - nearest hotel: Club Recreativo Armada de Chile


,Country,Lat,Lng,City,Humidity,Hotel Name
0,AR,-38.0023,-57.5575,mar del plata,56,Nuevo Ostende
1,LY,29.1614,16.1390,waddan,9,No hotel found
148,BR,-29.6842,-53.8069,santa maria,30,Hotel Morotin
159,ES,28.6585,-17.9182,los llanos de aridane,41,Valle Aridane
233,EG,25.2934,32.5540,esna,15,No hotel found
296,MA,30.4202,-9.5982,agadir,40,Atlantic
343,US,42.2506,-75.8330,broome,41,No hotel found
360,IR,25.2919,60.6430,chabahar,67,هتل سپیده
369,MM,19.6742,97.2094,loikaw,40,Nawaday Hotel
389,MX,30.7000,-111.8167,altar,20,Hotel Altar Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_height = 600,
    size = "Humidity",
    scale = 1.5,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)


# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)